In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.plotly as py
import plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

plt.style.use('bmh')
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
init_notebook_mode(connected=True) 

data = pd.read_csv('summer19.csv')


def comma_to_point(x):
    return x.replace(',','.')

#data['gamelength'] = data['gamelength'].apply(comma_to_point).astype('float64')
#data['fttime'] = data['fttime'].apply(comma_to_point).astype('float64')

data.head(12)

,gameid,url,league,split,date,week,game,patchno,playerid,side,...,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
0,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",1,Blue,...,-668.0,4453.0,5258.0,-805.0,72.0,89.0,-17.0,127.0,152.0,-25.0
1,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",2,Blue,...,546.0,3876.0,3255.0,621.0,69.0,60.0,9.0,110.0,93.0,17.0
2,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",3,Blue,...,402.0,4973.0,4549.0,424.0,83.0,86.0,-3.0,139.0,145.0,-6.0
3,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",4,Blue,...,406.0,3275.0,3002.0,273.0,95.0,67.0,28.0,148.0,122.0,26.0
4,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",5,Blue,...,-15.0,2683.0,2557.0,126.0,15.0,14.0,1.0,24.0,23.0,1.0
5,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",6,Red,...,668.0,5258.0,4453.0,805.0,89.0,72.0,17.0,152.0,127.0,25.0
6,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",7,Red,...,-546.0,3255.0,3876.0,-621.0,60.0,69.0,-9.0,93.0,110.0,-17.0
7,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",8,Red,...,-402.0,4549.0,4973.0,-424.0,86.0,83.0,3.0,145.0,139.0,6.0
8,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",9,Red,...,-406.0,3002.0,3275.0,-273.0,67.0,95.0,-28.0,122.0,148.0,-26.0
9,1092338,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2019-2,"43617,64992",1,1,"9,1",10,Red,...,15.0,2557.0,2683.0,-126.0,14.0,15.0,-1.0,23.0,24.0,-1.0


In [ ]:
player_names = data.player.unique()

# Initialize the Dataframe
players = pd.DataFrame(data[(data.player == 'Licorice')].mean())
players = players.transpose()
players.rename(index={0:'Licorice'}, inplace=True)

# Append each player
for player in player_names:
    if player != 'Team' and player != 'Licorice':
        p = pd.DataFrame((data[(data.player == player)].mean())).transpose()
        p.rename(index={0: player}, inplace=True)
        players = pd.concat([players, p], sort=False)
        
# Establish players mean max min median might be useful later
player_mean = pd.DataFrame(players.mean()).transpose()
player_max = pd.DataFrame(players.max()).transpose()
player_min = pd.DataFrame(players.min()).transpose()
player_median = pd.DataFrame(players.median()).transpose()

# Wards, dmg, KDA, Kill participation
# Gold diff at 15, Total damage, Gold spread
labels = ['wards','dmgtochamps','totalgold','k','d','a','gdat15']

# update player positions as they had been lost
players.insert(len(players.columns), column='position',value='ADC')
players.insert(len(players.columns), column='team',value='Cloud9')
for player in player_names:
    if player != 'Team':
        players.loc[player, 'position'] = data[data.player == player].position.values[0]
        players.loc[player, 'team'] = data[data.player == player].team.values[0]


team_names = data.team.unique()
teams = pd.DataFrame(data[(data.player == 'Team') & (data.team == 'Cloud9')].mean())
teams = teams.transpose()
teams.rename(index={0:'Cloud9'}, inplace=True)

for team in team_names:
    if team != 'Cloud9':
        p = pd.DataFrame((data[(data.team == team) & (data.player == 'Team')].mean())).transpose()
        p.rename(index={0: team}, inplace=True)
        teams = pd.concat([teams, p], sort=False)

In [ ]:
# Used for data visualization later
def simple_radar_graph(player_name, color='blue', players=players, role='ADC'):
    x = pd.DataFrame(players.loc[player_name, labels]).transpose()
    player_max = pd.DataFrame(players[player.position == role].max()).transpose()
    
    data = [go.Scatterpolar(
      r = [(x['wards'].values[0] / player_max['wards'].values[0]) * 100,(x['dmgtochamps'].values[0] / player_max['dmgtochamps'].values[0]) * 100,100 * (x['totalgold'].values[0] / player_max['totalgold'].values[0]),100 * (x['k'].values[0] / player_max['k'].values[0]),100 * (x['d'].values[0] / player_max['d'].values[0]),100 * (x['a'].values[0] / player_max['a'].values[0]),100 * (x["gdat15"].values[0] / player_max['gdat15'].values[0]), 100 * (x["wards"].values[0] / player_max['wards'].values[0])],
      theta = ['Wards','Damages to champs', 'Total gold','Kills','Deaths','Assists','Opponent gold difference at 15m','Wards'],
      fill = 'toself',
         line =  dict(
                color = color
            )
    )]

    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, 100]
        )
      ),
      showlegend = False,
      title = "{}".format(player_name)
    )
    fig = go.Figure(data=data, layout=layout)
    

    return iplot(fig, filename = "Radar")

# used for data visualization later
def vs_radar_graph(player_name1, player_name2, color1='orange', color2='lightgray',title=None, players=players,role='ADC'):
    player_max = pd.DataFrame(players[players.position == role].max()).transpose()
    
    if not title:
         title='{} vs {}'.format(player_name1, player_name2)
    
    x1 = pd.DataFrame(players.loc[player_name1, labels]).transpose()
    x2 = pd.DataFrame(players.loc[player_name2, labels]).transpose()
    # MEDIAN : x3 = pd.DataFrame(players[players.position == role].median()).transpose()
    
    """
        MEDIAN
        go.Scatterpolar(
      name = player_name2,
      r = [(x3['wards'].values[0] / player_max['wards'].values[0]) * 100,(x3['dmgtochamps'].values[0] / player_max['dmgtochamps'].values[0]) * 100,100 * (x3['totalgold'].values[0] / player_max['totalgold'].values[0]),100 * (x3['k'].values[0] / player_max['k'].values[0]),100 * (x3['d'].values[0] / player_max['d'].values[0]),100 * (x3['a'].values[0] / player_max['a'].values[0]),100 * (x3["wards"].values[0] / player_max['wards'].values[0])],
      theta = ['Wards','Damages to champs', 'Total gold','Kills','Deaths','Assists','Wards'],
      fill = 'toself',
         line =  dict(
                color = 'black'
            )
        ),
        
    add as 1st element in array below if you want
    """
    
    data = [go.Scatterpolar(
      name = player_name1,
      r = [(x1['wards'].values[0] / player_max['wards'].values[0]) * 100,(x1['dmgtochamps'].values[0] / player_max['dmgtochamps'].values[0]) * 100,100 * (x1['totalgold'].values[0] / player_max['totalgold'].values[0]),100 * (x1['k'].values[0] / player_max['k'].values[0]),100 * (x1['d'].values[0] / player_max['d'].values[0]),100 * (x1['a'].values[0] / player_max['a'].values[0]),100 * (x1["gdat15"].values[0] / player_max['gdat15'].values[0]),100 * (x1["wards"].values[0] / player_max['wards'].values[0])],
      theta = ['Wards','Damages to champs', 'Total gold','Kills','Deaths','Assists','Opponent gold difference at 15m','Wards'],
      fill = 'toself',
         line =  dict(
                color = color1
            )
    ),
           
    go.Scatterpolar(
      name = player_name2,
      r = [(x2['wards'].values[0] / player_max['wards'].values[0]) * 100,(x2['dmgtochamps'].values[0] / player_max['dmgtochamps'].values[0]) * 100,100 * (x2['totalgold'].values[0] / player_max['totalgold'].values[0]),100 * (x2['k'].values[0] / player_max['k'].values[0]),100 * (x2['d'].values[0] / player_max['d'].values[0]),100 * (x2['a'].values[0] / player_max['a'].values[0]),100 * (x2["gdat15"].values[0] / player_max['gdat15'].values[0]),100 * (x2["wards"].values[0] / player_max['wards'].values[0])],
      theta = ['Wards','Damages to champs', 'Total gold','Kills','Deaths','Assists','Opponent gold difference at 15m','Wards'],
      fill = 'toself',
         line =  dict(
                color = color2
            )
    )
           
    ]

    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, 100]
        )
      ),
      showlegend = False,
      title = "{}".format(title)
    )
    fig = go.Figure(data=data, layout=layout)
    

    return iplot(fig, filename = "Radar")

teams_labels = ['fb','gamelength', 'k', 'd', 'dmgtochamps','totalgold','fbaron','herald','ft','firstmidouter','firsttothreetowers','gdat15']
# used for data visualization later
def teamvs_radar_graph(team_name1, team_name2, color1='orange', color2='lightgray',title=None, teams=teams):
    team_max = pd.DataFrame(teams.max()).transpose()
    
    if not title:
         title='{} vs {}'.format(team_name1, team_name2)
    
    x1 = pd.DataFrame(teams.loc[team_name1, teams_labels]).transpose()
    x2 = pd.DataFrame(teams.loc[team_name2, teams_labels]).transpose()

    data = [go.Scatterpolar(
      name = team_name1,
      r = [(x1['fb'].values[0] / team_max['fb'].values[0]) * 100,(x1['gamelength'].values[0] / team_max['gamelength'].values[0]) * 100,(x1['k'].values[0] / team_max['k'].values[0]) * 100,(x1['d'].values[0] / team_max['d'].values[0]) * 100,(x1['dmgtochamps'].values[0] / team_max['dmgtochamps'].values[0]) * 100,100 * (x1['totalgold'].values[0] / team_max['totalgold'].values[0]),100 * (x1['fbaron'].values[0] / team_max['fbaron'].values[0]),100 * (x1['herald'].values[0] / team_max['herald'].values[0]),100 * (x1['ft'].values[0] / team_max['ft'].values[0]),100 * (x1["firstmidouter"].values[0] / team_max['firstmidouter'].values[0]),100 * (x1["firsttothreetowers"].values[0] / team_max['firsttothreetowers'].values[0]),100 * (x1["gdat15"].values[0] / team_max['gdat15'].values[0]),100 * (x1["fb"].values[0] / team_max['fb'].values[0])],
      theta =['First Blood', 'Game length', 'Kills', 'Deaths', 'Damages to champs','Total gold','First Baron','Herald','First tower','First mid outer tower','First to three towers','Gold difference at 15', 'First Blood'],
      fill = 'toself',
         line =  dict(
                color = color1
            )
    ),
           
    go.Scatterpolar(
      name = team_name2,
      r = [(x2['fb'].values[0] / team_max['fb'].values[0]) * 100,(x2['gamelength'].values[0] / team_max['gamelength'].values[0]) * 100,(x2['k'].values[0] / team_max['k'].values[0]) * 100, (x2['d'].values[0] / team_max['d'].values[0]) * 100,(x2['dmgtochamps'].values[0] / team_max['dmgtochamps'].values[0]) * 100,100 * (x2['totalgold'].values[0] / team_max['totalgold'].values[0]),100 * (x2['fbaron'].values[0] / team_max['fbaron'].values[0]),100 * (x2['herald'].values[0] / team_max['herald'].values[0]),100 * (x2['ft'].values[0] / team_max['ft'].values[0]),100 * (x2["firstmidouter"].values[0] / team_max['firstmidouter'].values[0]),100 * (x2["firsttothreetowers"].values[0] / team_max['firsttothreetowers'].values[0]),100 * (x2["gdat15"].values[0] / team_max['gdat15'].values[0]),100 * (x2["fb"].values[0] / team_max['fb'].values[0])],
      theta =['First Blood', 'Game length', 'Kills', 'Deaths', 'Damages to champs','Total gold','First Baron','Herald','First tower','First mid outer tower','First to three towers','Gold difference at 15', 'First Blood'],
      fill = 'toself',
         line =  dict(
                color = color2
            )
    )
           
    ]

    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, 100]
        )
      ),
      showlegend = False,
      title = "{}".format(title)
    )
    fig = go.Figure(data=data, layout=layout)
    

    return iplot(fig, filename = "Radar")

In [ ]:
players_corr = players

# to avoid running the kernel everytime
if 'gameid' in players_corr.columns.tolist():
    # doubles/triples/quadras/pentas/okpm have suffered damages with the .mean() for unknown reasons,
    # elementals, airdrakes, ... are chosen with randomness so they're not significant
    # heraldtime barontime are not contained in players variables
    # time var : make no sense in corr
    players_corr = players_corr.drop(['doubles','triples','quadras','pentas','gameid','playerid','firedrakes','elders','elementals','waterdrakes','earthdrakes','airdrakes','heraldtime','fbarontime','okpm', 'fttime', 'gamelength'], axis=1)

corr = players_corr.corr()
f, ax = plt.subplots(figsize=(24, 16))
heatmap = sns.heatmap(corr)